In [1]:
"""setup"""
import pandas as pd
import pytest
import sys
sys.path.append('../..')
from altair_widgets import interact_with
from altair import load_dataset

'setup'

In [2]:
"""test_input_fail"""
d = [{'a': 1, 'b': 2, 'c': 4}]*4

with pytest.raises(ValueError):
    interact_with(d)

'test_input_fail'

In [3]:
"""test_input"""
df = pd.DataFrame(d)
interact_with(df, show=False)

'test_input'

In [4]:
"""test_basic_config"""
df = load_dataset('iris')
df.head()
w = interact_with(df, show=False)
w.settings['encodings'] = [{'encoding': 'x', 'field': 'petalWidth'},
                           {'encoding': 'y', 'field': 'petalLength'},
                           {'encoding': 'color', 'field': 'species'}]
w.plot(show=False)

'test_basic_config'

,petalLength,petalWidth,sepalLength,sepalWidth,species
0,1.4,0.2,5.1,3.5,setosa
1,1.4,0.2,4.9,3.0,setosa
2,1.3,0.2,4.7,3.2,setosa
3,1.5,0.2,4.6,3.1,setosa
4,1.4,0.2,5.0,3.6,setosa


In [5]:
"""test_generates_same_chart_as_altair"""
import altair
from altair import Chart

c = Chart(df).mark_point().encode(
    x='petalWidth', y='petalLength', color='species')

assert type(w.chart) is altair.api.Chart
assert c.to_dict() == w.chart.to_dict()

'test_generates_same_chart_as_altair'

In [7]:
"""test_bg_color"""
w = interact_with(df, show=False)
w.settings = {'mark': {'mark': 'mark_text', 'applyColorToBackground': True},
              'encodings': [{'encoding': 'row', 'field': 'species'},
                            {'encoding': 'color', 'field': 'petalLength'},
                            {'encoding': 'text', 'field': ' '}]
              }
w.plot(show=False)
c = Chart(df).mark_text(applyColorToBackground=True).encode(
    row='species',
    color='petalLength',
    text=' ')
    
assert c.to_dict() == w.chart.to_dict()

'test_bg_color'

In [8]:
"""test_log"""
w = interact_with(df, show=False)
w.settings['encodings'] = [{'encoding': 'x', 'field': 'petalWidth', 'scale': 'log'},
                           {'encoding': 'y', 'field': 'petalLength'},
                           {'encoding': 'color', 'field': 'species'}]
w.plot(show=False)
c = Chart(df).mark_point().encode(
    x=altair.X('petalWidth', scale=altair.Scale(type='log')),
    y='petalLength', color='species')
    
assert c.to_dict() == w.chart.to_dict()

'test_log'

In [9]:
"""test_update"""
class Event:
    def __init__(self, row, title, value):
        self.row = row
        self.title = title
        self.value = value

w = interact_with(df, show=False)
w.settings['encodings'] = [{'encoding': 'x', 'field': 'petalWidth', 'scale': 'log'},
                           {'encoding': 'y', 'field': 'petalLength'},
                           {'encoding': 'color', 'field': 'species'}]
old = 'linear'
event = {'owner': Event(1, 'scale', old), 'old': old, 'new': 'log'}
w._update(event)
new_settings = [{'encoding': 'x', 'field': 'petalWidth', 'scale': 'log'},
                {'encoding': 'y', 'field': 'petalLength', 'scale': 'log'},
                {'encoding': 'color', 'field': 'species'}]

assert new_settings == w.settings['encodings']

'test_update'

In [11]:
"""test_adding_dimension"""
w = interact_with(df, show=False)
w.settings['encodings'] = [{'encoding': 'x', 'field': 'petalWidth'},
                           {'encoding': 'y', 'field': 'petalLength'},
                           {'encoding': 'color', 'field': 'species'}]
w._add_dim(None)
w.settings['encodings'][-1]['encoding'] = 'size'
w.settings['encodings'][-1]['field'] = 'sepalLength'
w.plot(w.settings)

c = Chart(df).mark_point().encode(
    x='petalWidth', y='petalLength', color='species', size='sepalLength'
)

assert w.chart.to_dict() == c.to_dict()

'test_adding_dimension'

In [12]:
"""test_ndims"""
w = interact_with(df, ndims=1, show=False)
assert len(w.settings['encodings']) == 1

'test_ndims'